In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import h5py
import healpy as hp
import kalepy as kale

import holodeck as holo
import holodeck.anisotropy as anis
from holodeck import detstats, plot, utils
from holodeck.constants import YR, MSOL, GYR

from matplotlib.lines import Line2D

In [ ]:
SHAPE=(91,81,101)
SAVE=True

# Setup Example Model

In [ ]:
# get frequencies
dur = holo.librarian.DEF_PTA_DUR*YR
fobs_gw_cents, fobs_gw_edges = holo.utils.pta_freqs(dur)
fobs_orb_cents = fobs_gw_cents/2
fobs_orb_edges = fobs_gw_edges/2

In [ ]:
# build Semi-Analytic-Model
sam = holo.sams.Semi_Analytic_Model(shape=SHAPE)
hard = holo.hardening.Fixed_Time_2PL_SAM(sam, 3*GYR)
if isinstance(hard, holo.hardening.Fixed_Time_2PL_SAM):
    hard_name = '%dGyr Fixed Time' % (hard._target_time/GYR)
elif isinstance(hard, holo.hardening.Hard_GW):
    hard_name = 'GW Only'
print(hard_name)

In [ ]:

# Evolve model to get final number densities & redshifts at bin edges
redz_final, dnum = holo.sam_cython.dynamic_binary_number_at_fobs(
    fobs_orb_cents, sam, hard, holo.cosmo)

# calculate final number per bin at bin centers
edges = [sam.mtot, sam.mrat, sam.redz, fobs_orb_edges]
number = holo.sam_cython.integrate_differential_number_3dx1d(edges, dnum)
intnum = np.floor(number)

# format redz_prime and redz_initial for use like redz_final
redz_prime = sam._redz_prime[...,np.newaxis]
redz_initl = np.array(sam.redz)[np.newaxis,np.newaxis,:,np.newaxis]
mm = len(edges[0])
qq = len(edges[1])
zz = len(edges[2])
redz_initl = np.repeat(redz_initl, mm*qq).reshape(mm,qq,zz,1)
print(f"{redz_final.shape=}, {redz_prime.shape=}, {redz_initl.shape=}")



In [ ]:
# calculate strain amplitudes

# at bin centers
hs_cents_initl = anis.strain_amp_at_bin_centers_redz(edges, None)
hs_cents_prime = anis.strain_amp_at_bin_centers_redz(edges, redz_prime)
hs_cents_final = anis.strain_amp_at_bin_centers_redz(edges, redz_final) 
print(f"{hs_cents_initl.shape=}, {hs_cents_prime.shape=}, {hs_cents_final.shape=}")

# at bin edges
hs_edges_initl = anis.strain_amp_at_bin_edges_redz(edges, None)
hs_edges_prime = anis.strain_amp_at_bin_edges_redz(edges, redz_prime)
hs_edges_final = anis.strain_amp_at_bin_edges_redz(edges, redz_final) 
print(f"{hs_edges_initl.shape=}, {hs_edges_prime.shape=}, {hs_edges_final.shape=}")

# Calculate Cl's from Number (bin centers)

In [ ]:
# from number and hs at bin centers
C0_cents_initl, Cl_cents_initl, = anis.Cl_analytic_from_num(fobs_orb_edges, number, hs_cents_initl)
C0_cents_prime, Cl_cents_prime, = anis.Cl_analytic_from_num(fobs_orb_edges, number, hs_cents_prime)
C0_cents_final, Cl_cents_final, = anis.Cl_analytic_from_num(fobs_orb_edges, number, hs_cents_final)

In [ ]:
# from number and hs at bin centers
C0_floor_initl, Cl_floor_initl, = anis.Cl_analytic_from_num(fobs_orb_edges, intnum, hs_cents_initl)
C0_floor_prime, Cl_floor_prime, = anis.Cl_analytic_from_num(fobs_orb_edges, intnum, hs_cents_prime)
C0_floor_final, Cl_floor_final, = anis.Cl_analytic_from_num(fobs_orb_edges, intnum, hs_cents_final)

In [ ]:
fig = anis.plot_ClC0_versions(fobs_gw_cents, spk=True, bayes=False, sim=False)
ax = fig.axes[0]
ax.set_title('%s, shape=%s' % (hard_name, str(sam.shape)))

anis.draw_bayes(ax, lmax=6, ms=4)


C0_cents = [C0_cents_initl, C0_cents_prime, C0_cents_final]
Cl_cents = [Cl_cents_initl, Cl_cents_prime, Cl_cents_final]

C0_floor = [C0_floor_initl, C0_floor_prime, C0_floor_final]
Cl_floor = [Cl_floor_initl, Cl_floor_prime, Cl_floor_final]

lw = [1,1.5,2]
colors = ['goldenrod', 'tab:orange', 'tab:red',]
labels = ['init', 'prime', 'final']
handles = []
for ii in range(3):
    print(ii)
    hh = anis.draw_analytic(ax, C0=C0_cents[ii],Cl=Cl_cents[ii], fobs_gw_cents=fobs_gw_cents,
                    label=labels[ii], ls='-.', color=colors[ii], lw=lw[ii])
    anis.draw_analytic(ax, C0=C0_floor[ii],Cl=Cl_floor[ii], fobs_gw_cents=fobs_gw_cents,
                    label=None, ls=':', color=colors[ii], lw=lw[ii])
    handles.append(hh)
leg = ax.legend(handles=handles, loc='lower left')

leg_handles = [
    Line2D([0], [0], label='SPK', ls='--', color='limegreen'),
    Line2D([0], [0], label='upper lims', ls='-', marker='o', ms=4, color='k'),
    Line2D([0], [0], label='cents number', ls='-.', color=colors[0]),
           Line2D([0], [0], label='cents rounded', ls=':', color=colors[0])]
leg2 = ax.legend(handles=leg_handles, loc='lower right')
ax.add_artist(leg)


# Calculate Cl's from dnum (bin edges)

In [ ]:
C0_edges_initl, Cl_edges_initl = anis.Cl_analytic_from_dnum(edges, dnum, redz_initl)
C0_edges_prime, Cl_edges_prime = anis.Cl_analytic_from_dnum(edges, dnum, redz_prime)
C0_edges_final, Cl_edges_final = anis.Cl_analytic_from_dnum(edges, dnum, redz_final)

In [ ]:
fig = anis.plot_ClC0_versions(fobs_gw_cents, spk=True, bayes=False, sim=False)
ax = fig.axes[0]
ax.set_title('%s, shape=%s' % (hard_name, str(sam.shape)))

anis.draw_bayes(ax, lmax=6, ms=4, colors=['k', 'k', 'k', 'k', 'k', 'k'])

Cl_edges = [Cl_edges_initl, Cl_edges_prime, Cl_edges_final]
C0_edges = [C0_edges_initl, C0_edges_prime, C0_edges_final]

handles = []
for ii in range(3):
    print(ii)
    hh = anis.draw_analytic(ax, C0=C0_edges[ii], Cl=Cl_edges[ii], fobs_gw_cents=fobs_gw_cents,
                       label=labels[ii], ls='-', color=colors[ii], lw=lw[ii])
    anis.draw_analytic(ax, C0=C0_floor[ii],Cl=Cl_floor[ii], fobs_gw_cents=fobs_gw_cents,
                    label=None, ls=':', color=colors[ii], lw=lw[ii])
    anis.draw_analytic(ax, C0=C0_cents[ii],Cl=Cl_cents[ii], fobs_gw_cents=fobs_gw_cents,
                    label=labels[ii], ls='-.', color=colors[ii], lw=lw[ii])
    handles.append(hh)

leg1 = ax.legend(handles=handles, loc='lower left')

leg_handles = [
    Line2D([0], [0], label='edges dnum', ls='-', color=colors[-1]),
    Line2D([0], [0], label='cents number', ls='-.', color=colors[-1]),
    Line2D([0], [0], label='cents rounded', ls=':', color=colors[-1]),
    Line2D([0], [0], label='SPK', ls='--', color='limegreen'),
    Line2D([0], [0], label='bayes upper lims', ls='-', marker='o', ms=4, color='k'),
]
leg2 = ax.legend(handles=leg_handles, loc='lower right', ncols=2)
ax.add_artist(leg1)


Make sure that prime and initl aren't the exact same

In [ ]:
print(holo.utils.stats(Cl_edges_initl))
print(holo.utils.stats(Cl_edges_prime))

In [ ]:
print(holo.utils.stats(Cl_cents_prime))
print(holo.utils.stats(Cl_cents_initl))

# Calculate Cl's from Random Realizations 
(always cents)

In [ ]:
C0_reals_initl, Cl_reals_initl = anis.Cl_analytic_from_dnum(edges, dnum, redz_initl, realize=20)
print('one done')
C0_reals_prime, Cl_reals_prime = anis.Cl_analytic_from_dnum(edges, dnum, redz_prime, realize=20)
print('two done')
C0_reals_final, Cl_reals_final = anis.Cl_analytic_from_dnum(edges, dnum, redz_final, realize=20)


In [ ]:
print(C0_reals_initl.shape)

In [ ]:
fig = anis.plot_ClC0_versions(fobs_gw_cents, spk=True, bayes=False, sim=False)
ax = fig.axes[0]
ax.set_title('%s, shape=%s' % (hard_name, str(sam.shape)))

anis.draw_bayes(ax, lmax=6, ms=4, colors=['k', 'k', 'k', 'k', 'k', 'k'])
labels = ['z_initial', 'z_prime', 'z_final']
colors=['olive', '#ff4000', 'deeppink']
lw = [2, 2.5, 3]

C0_reals = [C0_reals_initl, C0_reals_prime, C0_reals_final]
Cl_reals = [Cl_reals_initl, Cl_reals_prime, Cl_reals_final]

handles = []
for ii in range(3):
    print(ii)
    hh = anis.draw_analytic(ax, C0=C0_edges[ii], Cl=Cl_edges[ii], fobs_gw_cents=fobs_gw_cents,
                       label=labels[ii], ls='-', color=colors[ii], lw=lw[ii], alpha=0.7)
    anis.draw_analytic(ax, C0=C0_floor[ii],Cl=Cl_floor[ii], fobs_gw_cents=fobs_gw_cents,
                    label=None, ls='--', color=colors[ii], lw=lw[ii], alpha=0.5)
    anis.draw_analytic(ax, C0=C0_cents[ii],Cl=Cl_cents[ii], fobs_gw_cents=fobs_gw_cents,
                    label=labels[ii], ls='-.', color=colors[ii], lw=lw[ii], alpha=0.7)
    anis.draw_reals(ax, Cl_reals[ii], C0_reals[ii], fobs_gw_cents,
                    color=colors[ii], label=None, show_ci=True, show_reals=True, nshow=20)
    handles.append(hh)

leg1 = ax.legend(handles=handles, loc='lower right', bbox_to_anchor=(1,0.2))

leg_handles = [
    Line2D([0], [0], label='edges dnum', ls='-', color=colors[-1], alpha=0.7),
    Line2D([0], [0], label='cents number', ls='-.', color=colors[-1], alpha=0.7),
    Line2D([0], [0], label='cents rounded', ls='--', color=colors[-1], alpha=0.5),
    Line2D([0], [0], label='realizations', ls=':', color=colors[-1], alpha=0.5),
    Line2D([0], [0], label='SPK', ls='--', color='limegreen'),
    Line2D([0], [0], label='bayes upper lims', ls='-', marker='o', ms=4, color='k'),
]
leg2 = ax.legend(handles=leg_handles, loc='lower right', ncols=2)
ax.add_artist(leg1)

fig.tight_layout()

if SAVE:
# if True:
    fig_path = '/Users/emigardiner/GWs/holodeck/output/figures/anisotropy'
    name = fig_path+f'/anlt_{hard._target_time/GYR}GyrFT_{sam.shape[0]}_{sam.shape[1]}_{sam.shape[2]}.png'
    fig.savefig(name, bbox_inches='tight', dpi=100)


In [ ]:
print(holo.utils.stats(redz_initl))
print(holo.utils.stats(redz_prime))
print(holo.utils.stats(redz_final))


In [ ]:
print(holo.utils.stats(Cl_cents_final))
print(holo.utils.stats(Cl_cents_prime))
print(holo.utils.stats(Cl_cents_initl))

In [ ]:
print(holo.utils.stats(Cl_floor_final))
print(holo.utils.stats(Cl_floor_prime))
print(holo.utils.stats(Cl_floor_initl))